In [1]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part, FinishReason
import vertexai.preview.generative_models as generative_models
import os
import json
import pandas as pd

In [2]:
with open('gemini-ml-esg-sentiment/prompts.json', encoding='utf-8') as arq:
        prompts = json.load(arq)

prompt_string = prompts["prompts"]["prompt_positive_negative_investor_context_en"]

In [3]:
dataset = pd.read_csv('gemini-ml-esg-sentiment/ML-ESG-2_English_Train_formatted.csv')
dataset_to_classify = dataset['text'].tolist()

In [4]:
def batches(lista, tamanho_lote):
    for i in range(0, len(lista), tamanho_lote):
        yield lista[i:i + tamanho_lote]

In [14]:
safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
}

generation_config = {
    "max_output_tokens": 40,
    "temperature": 0,
    "top_p": 0.95,
}

def generate(prompt_instance):
  vertexai.init(project="aida-412720", location="us-central1")
  model = GenerativeModel("gemini-1.0-pro")
  responses = model.generate_content(
      [prompt_instance],
      generation_config=generation_config,
      safety_settings=safety_settings,
      stream=True,
  )
  response_string = ""
  for response in responses:
    response_string = response_string + response.text
    
  return response_string

In [15]:

prompt_instance = prompt_string + dataset_to_classify[1]
respons = generate(prompt_instance)
print(prompt_instance)
print(respons)

Evaluate the sentiment conveyed in the forthcoming finance-related passage through the lens of an investor. Determine whether it is 'Positive' or 'Negative': The company also announced the appointment of tim wong as its new chief product officer. joining from finbourne technology, where he oversaw product development for investment and portfolio management software, wong also holds several years of experience in product management and strategy consulting at companies including amazon, capco, mckinsey and pwc.
## Sentiment Analysis: Positive 

**Reasons for the positive sentiment:**

* **New Chief Product Officer:** The appointment of Tim Wong as the new


In [16]:
all_responses = []
i=1
for batch in batches(dataset_to_classify, 100):
    for item in batch:
        prompt_instance = prompt_string + item
        resp = generate(prompt_instance)
        all_responses.append(resp)
        
    print("batch number ", i)
    i+=1
    

batch number  1
batch number  2
batch number  3
batch number  4
batch number  5
batch number  6
batch number  7
batch number  8
batch number  9


In [17]:
print(all_responses)

['## Sentiment Analysis: Positive \n\n**Reasons for Positive Sentiment:**\n\n* **Appointment of a new CEO:**  Leadership changes can be a positive sign', '## Sentiment Analysis: Positive \n\n**Reasons for Positive Sentiment:**\n\n* **New Chief Product Officer:** The appointment of a new Chief Product Officer (', '## Sentiment Analysis: Positive \n\n**Reasons for Positive Sentiment:**\n\n* **Problem-Solution Approach:** Wong identifies a problem (complexities of personalized', '## Sentiment Analysis: \n\n**Positive** \n\n**Reasons:**\n\n* **Increased demand for renewables and non-fossil fuels:** The report highlights', "## Sentiment Analysis: Positive \n\nThe passage conveys a **positive sentiment** for investors. Here's why:\n\n* **New", '## Positive \n\n**Reasons:**\n\n* **Green steel production:** The technology enables the production of steel without CO2 emissions, aligning with the growing', '## Sentiment Analysis: Positive \n\nThe provided passage expresses a **positive sentiment*

In [18]:
df_predictions = pd.DataFrame({
    "text": dataset['text'].tolist(),
    "label": dataset['label'].tolist(),
    'response': all_responses,

})

df_predictions.to_csv('gemini-ml-esg-sentiment/ML-ESG-2_English_Train_formatted_responses_pos_neg_investor.csv', index=False)

In [19]:
def detect_sentiment_word(text):
    text_lower = text.lower()
    if "positive" in text_lower:
        return "positive"
    elif "negative" in text_lower:
        return "negative"
    else:
        return "undetermined" 

In [20]:
responses_label = []

for response in all_responses:
    resp_label = detect_sentiment_word(response)
    responses_label.append(resp_label)

In [21]:
df_predictions_label = pd.DataFrame({
    "text": dataset['text'].tolist(),
    "label": dataset['label'].tolist(),
    'response': all_responses,
    'responseLabel': responses_label,

})

df_predictions_label.to_csv('gemini-ml-esg-sentiment/ML-ESG-2_English_Train_formatted_responses_pos_neg_investor_with_label.csv', index=False)

In [22]:

responses = pd.read_csv('gemini-ml-esg-sentiment/ML-ESG-2_English_Train_formatted_responses_pos_neg_investor_with_label.csv')

frequency_matrix = pd.crosstab(index=[responses['label'], responses['responseLabel']], columns='count')

frequency_matrix

col_0                      count
label       responseLabel       
Opportunity negative          31
            positive         661
            undetermined       2
Risk        negative          97
            positive          16
            undetermined       1